In [4]:
start_index = 0
end_index = 215
criteria = 'standing_desk'
model = 'gpt-4.1-2025-04-14'
template = 'standing_desk'
file_path = './buffer/nee/to_an.csv'

to_analyze = True
to_translate_comment = False
to_translate_analysis = False
to_check = False

In [ ]:
import pandas as pd

df = pd.read_csv(file_path, sep=None, engine='python')
print(df.columns)
print(df.shape)


if True:
    # df = df[['评论内容', '评分', '评论时间']]
    df['英文分析'] = ''
    df['好差评结果'] = ''
    df['中文评论'] = ''
    df['中文分析'] = ''
    df['审查结果'] = ''

Index(['﻿ss', 'product_handle', '评分', 'title', 'author', 'email', '评论内容',
       'created_at', 'avatar', 'country_code', 'status', 'Unnamed: 11',
       'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15',
       'Unnamed: 16'],
      dtype='object')
(215, 17)


In [6]:
import re
import yaml
from tqdm import tqdm
from modules.agent import OpenAICommentAnalysisAgent



def extract_output(text):
    pattern = r'\{.*?\}'
    matches = re.findall(pattern, text, re.DOTALL)

    matches = [match.replace('\n', '') for match in matches]
    return matches


def remove_duplicate_values(input_dict):
    seen_values = set()  # 用于存储已经出现过的值
    output_dict = {}  # 用于存储最终的结果

    for key, value in input_dict.items():
        if value not in seen_values:
            seen_values.add(value)
            output_dict[key] = value

    return output_dict


with open('./openai_keys.yaml', "r", encoding="utf-8") as file:
    data = yaml.safe_load(file)
my_key = data['openai']['key']

comment_analyzor = OpenAICommentAnalysisAgent(openai_key=my_key, criteria=criteria, model=model, template=template)


for i in tqdm(range(start_index, end_index)):
    comment = df.iloc[i]['评论内容']
    if not isinstance(comment, str):
        continue

    # 分析评论
    if to_analyze:
        analysis = comment_analyzor.comment_analyze(comment)
        df.loc[i, '英文分析'] = analysis
        # print(f"分析结果: {analysis}")

        # 提取结果
        extracted_output = extract_output(analysis)
        extracted_output = ''.join(extracted_output)
        extracted_output = eval(extracted_output)
        extracted_output = remove_duplicate_values(extracted_output)
        df.loc[i, '好差评结果'] = str(extracted_output)
        # print(f"提取结果: {extracted_output}")

    # 翻译
    if to_translate_comment:
        translated_comment = comment_analyzor.translate(comment)
        df.loc[i, '中文评论'] = translated_comment
        # print(f"原: {comment}\n翻译评论: {translated_comment}")

    if to_translate_analysis:
        analysis = df.loc[i, '英文分析']
        translated_analysis = comment_analyzor.translate(analysis)
        df.loc[i, '中文分析'] = translated_analysis
        # print(f"翻译分析: {translated_analysis}")

    # 反审查
    if to_check:
        extracted_output = df.loc[i, '好差评结果']
        check_result = comment_analyzor.inspect(comment, str(extracted_output))
        df.loc[i, '审查结果'] = check_result
df.to_csv(file_path, sep='\t', index=False)

100%|██████████| 215/215 [12:17<00:00,  3.43s/it]


In [16]:
import pandas as pd


points = {'组装难度与时间': 'Assembly difficulty & Time', '说明书质量': 'Instruction Quality', '可定制性': 'Customizability', '组装部件数量': 'Number of Assembly Parts', '外观与风格': 'Appearance & Style', '颜色': 'Color', '桌面多板拼接': 'Multi-Board Desktop', '稳定性和坚固性': 'Stability & Sturdiness', '最高高度': 'Maximum Height', '高度调节顺畅度': 'Height adjustment smoothness', '高度调节速度': 'Height adjustment speed', '噪音水平': 'Noise Level', '预设高度功能': 'preset heights function', '防碰撞灵敏度': 'Anti-Collision Sensitivity', '模块化': 'Modularity', '照明功能': 'Lighting Feature', '集成电源和充电端口': 'Integrated Power and Charging Ports', '存储功能': 'Storage Features', '移动性': 'Mobility', '键盘托盘': 'Keyboard Tray', '产品可靠性': 'Product Reliability', '防水性': 'Water Resistance', '桌面厚度与材质': 'Desktop Quality', '五金件质量': 'Hardware Quality', '抽屉质量': 'Drawers Quality', '预钻孔对齐情况': 'Pilot Holes Alignment', '气味情况': 'Odor', '人体工学设计': 'ergonomic', '性价比': 'Value for Money', '发货速度': 'Shipping Speed', '包装质量': 'Packaging Quality', '产品完整性': 'Product Integrity', '客户服务': 'customer service', '售后服务': 'After-sales Service', '退换货体验': 'Return Experience', '线缆管理': 'Cable management', '桌面尺寸': 'Desktop Size', '桌子形状': 'Desk Shape', '承重能力': 'Load capacity', '值得推荐': 'Recommendation', '与商品图有差距': 'product image/description mismatch'}
points = {v.lower(): k for k, v in points.items()}

file_path = './buffer/nee/to_an.csv'
df = pd.read_csv(file_path, sep=None, engine='python')
df['好评'] = ''
df['差评'] = ''
df['中评'] = ''

for i in range(len(df)):
    ext_pts = df.iloc[i]['好差评结果']
    try:
        ext_pts = eval(ext_pts)
    except:
        print(ext_pts)
        continue
    pos = []
    neg = []
    neu = []
    for k in ext_pts.keys():
        if 'Pos' in k or 'pos' in k:
            ch = ext_pts[k]
            ch = ch.strip().lower()
            ch = points[ch]
            pos.append(ch)
        elif 'Neg' in k or 'neg' in k:
            ch = ext_pts[k]
            ch = ch.strip().lower()
            ch = points[ch]
            neg.append(ch)
        elif 'Neu' in k or 'neu' in k:
            ch = ext_pts[k]
            ch = ch.strip().lower()
            ch = points[ch]
            neu.append(ch)
    df.loc[i, '好评'] = str(pos)
    df.loc[i, '差评'] = str(neg)
    df.loc[i, '中评'] = str(neu)

df.to_csv(file_path, index=False, sep='\t')

nan
